# Data Modeling with Apache Cassandra database
________________
____________

## Introduction
In this Project we will help the **Sparkify** users to analyze their data by providing them a set of denormalized tables in **Apache Cassandra** database.
**Sparkify** has provided a collection of songs and user activity data of their new music straming app in the form of .csv files. 
Our goal is to read the source data and load them into Cassandra database with the help of ETL pipelines. 

Steps to follow:
* Read the source data (.csv files).
* Understand the user provided queries which are expected to run against Cassandra database.
* Create a KeySpace(database) in Cassandra.
* Create tables based on the user provided queries.
* Process the source files and create one final data file (.csv) file.
* Create ETL pipeline to load data from above created data file into Cassandra tables.
* Run the user expected queries to validate the queries.
______________

#### Import Python packages 

In [1]:
# In order to process the user requirements we need to import the following packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
#print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# for loop to loop through all files in a directory
for root, dirs, files in os.walk(filepath):
    # join filepath to the filename
    file_path_list = glob.glob(os.path.join(root,'*.csv'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a final data file by loading data from source files
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in our final data file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


___________________________
## Now we have a final data file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

Here is the screen shot of our data from our final data file <font color=red>**event_datafile_new.csv**</font>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Creating a cluster by connecting to our local instance
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Creating a *sparkify* Keyspace (database)

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy','replication_factor':1}"""
)
    
except Exception as e:
    print(e)

#### Connecting to above created keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Here are the requirements which are provided by *Sparkify* users
--------------

### Now we need to write CQL queries for the below requested user requirements.

* Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

* Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182  

* Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'  

______________________

In [8]:
# Final data file which we created in the above process.
file = 'event_datafile_new.csv'

#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

User planned to filter the data by sessionid and itemInSession.  
So our partition key would be *sessionId* and *itemInSession* would be clustering column to make the values unique in primary key.

In [9]:
# Here we drop the table if table exists and then create the table based on user requirement.
session.execute('DROP TABLE IF EXISTS SongDataBySession')
query = "CREATE TABLE IF NOT EXISTS SongDataBySession (sessionId int, ItemInSession int, artist text, song_title text, song_length float, PRIMARY KEY(sessionId,ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
# processing the final data file and inserting the data from required columns into target table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongDataBySession (sessionId,ItemInSession,artist,song_title,song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

In [11]:
# CQL query to view the expected output as per user requirement
query1 = ('select * from SongDataBySession where sessionId = 338 and ItemInSession=4')
rows = session.execute(query1)
for row in rows:
    print(row.sessionid,row.iteminsession,row.artist,row.song_title,round(row.song_length,4))

338 4 Faithless Music Matters (Mark Knight Dub) 495.3073


______________________

#### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

User planned to filter the data using userid and session id.  
So *userid and session* are our composite partition key and *itemInSession* is our clustering column to make the values unique in primary key.

In [12]:
# Here we drop the table if table exists and then create the table based on user requirement.
session.execute('DROP TABLE IF EXISTS SongDataByUser')
query2 = """CREATE TABLE IF NOT EXISTS SongDataByUser
                                                    (userid int
                                                    , sessionid int
                                                    , iteminsession int
                                                    , artist text
                                                    , song text
                                                    , username text
                                                    , PRIMARY KEY((userid,sessionid),iteminsession)
                                                    )"""
try:
    session.execute(query2)
except Exception as e:
    print(e)

In [13]:
# processing the final data file and inserting the data from required columns into target table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongDataByuser (userid,sessionid,iteminsession,artist,song,username) "
        query = query + "VALUES (%s, %s, %s, %s, %s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1]+' '+line[4]))

In [14]:
# CQL query to view the expected output as per user requirement
query21 = ('select artist,song,username from SongDataByUser where userid=10 and sessionid = 182')
rows = session.execute(query21)
for row in rows:
    print(row.artist,row.song,row.username)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


___________________________

#### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

User planned to filter the data through song title.  
*song* would be our partition key and *userid* would be our clustering column to make the values unique in primary key.

In [15]:
# Here we drop the table if table exists and then create the table based on user requirement.
session.execute('DROP TABLE IF EXISTS SongDataByTitle')
query3 = """CREATE TABLE IF NOT EXISTS SongDataByTitle
                                                    (song text
                                                    , username text
                                                    , userid int
                                                    , PRIMARY KEY(song,userid)
                                                    )"""
try:
    session.execute(query3)
except Exception as e:
    print(e)


In [16]:
# processing the final data file and inserting the data from required columns into target table.
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO SongDataByTitle (song,userid,username) "
        query = query + "VALUES (%s, %s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1]+' '+line[4]))

In [17]:
# CQL query to view the expected output as per user requirement
query31 = ("select username from SongDataByTitle where song ='All Hands Against His Own' ")
rows = session.execute(query31)
for row in rows:
    print(row.username)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Droping the tables before closing out the session.

In [18]:
session.execute('DROP TABLE IF EXISTS SongDataBySession')
session.execute('DROP TABLE IF EXISTS SongDataByUser')
session.execute('DROP TABLE IF EXISTS SongDataBySong')

### Closing Session and Cluster to exit

In [19]:
session.shutdown()
cluster.shutdown()